# Blue Brain BioExplorer

![](../bioexplorer_banner.png)

## Script to create a visualization of the ACE2 receptor

In [31]:
from bioexplorer import BioExplorer, Protein, Sugars, Quaternion, MovieMaker
import os

be = BioExplorer('localhost:5000')
be.reset_scene()
print(be.version())

1.2.1


In [32]:
# Resources
# resource_folder = '../../tests/test_files/'
resource_folder = '/home/favreau/git/BioExplorer/bioexplorer/pythonsdk/tests/test_files'
pdb_folder = os.path.join(resource_folder, 'pdb')
mesh_folder = os.path.join(resource_folder, 'obj')
complex_folder = os.path.join(resource_folder,'pdb','glycans','complex')
be.set_general_settings(logging_level=3, mesh_folder=mesh_folder)

protein_representation = be.REPRESENTATION_MESH
protein_radius_multiplier = 1.0

In [33]:
# Camera position
status = be.core_api().set_camera(
    orientation=[0.707, 0.0, -0.707, 0.0],
    position=[-25.772, 0.0, 0.005],
    target=[0.0, 0.0, 0.005]
)

In [34]:
# Protein
name = be.NAME_RECEPTOR
ace2_receptor = Protein(
    name=name,
    source=os.path.join(pdb_folder, '6m18.pdb'),
    rotation=Quaternion(0.707,-0.707,0,0)
)


status = be.add_protein(
    atom_radius_multiplier=protein_radius_multiplier,
    representation=protein_representation,
    protein=ace2_receptor
)

In [35]:
status = be.set_models_visibility(True)

## Protein information (AA sequences and glycosylation sites)

In [36]:
be.get_protein_amino_acid_information(assembly_name=name, name=name)

['5,768,MADYKDDDDKSGPDEVDASGRVRLVLPNPGLDARIPSLAELETIEQEEASSRPKWDNKAQYMLTCLGFCVGLGNVWRFPYLCQSHGGGAFMIPFLILLVLEGIPLLYLEFAIGQRLRRGSLGVWSSIHPALKGLGLASMLTSFMVGLYYNTIISWIMWYLFNSFQEPLPWSDCPLNENQTGYVDECARSSPVDYFWYRETLNISTSISDSGSIQWWMLLCLACAWSVLYMCTIRGIETTGKAVYITSTLPYVVLTIFLIRGLTLKGATNGIVFLFTPNVTELAQPDTWLDAGAQVFFSFSLAFGGLISFSSYNSVHNNCEKDSVIVSIINGFTSVYVAIVVYSVIGFRATQRYDDCFSTNILTLINGFDLPEGNVTQENFVDMQQRCNASDPAAYAQLVFQTCDINAFLSEAVEGTGLAFIVFTEAITKMPLSPLWSVLFFIMLFCLGLSSMFGNMEGVVVPLQDLRVIPPKWPKEVLTGLICLGTFLIGFIFTLNSGQYWLSLLDSYAGSIPLLIIAFCEMFSVVYVYGVDRFNKDIEFMIGHKPNIFWQVTWRVVSPLLMLIIFLFFFVVEVSQELTYSIWDPGYEEFPKSQKISYPNWVYVVVVIVAGVPSLTIPGYAIYKLIRNHCQKPGDHQGLVSTLSTASMNGDLKY',
 '5,768,MRSSSSWLLLSLVAVTAAWSHPQFEKQSTIEEQAKTFLDKFNHEAEDLFYQSSLASWNYNTNITEENVQNMNNAGDKWSAFLKEQSTLAQMYPLQEIQNLTVKLQLQALQQNGSSVLSEDKSKRLNTILNTMSTIYSTGKVCNPDNPQECLLLEPGLNEIMANSLDYNERLWAWESWRSEVGKQLRPLYEEYVVLKNEMARANHYEDYGDYWRGDYEVNGVDGYDYSRGQLIEDVEHTFEEIKPLYEHLHAYVRAKLMNAYPSYISPIGCLPAHLLGDMWGRFWTNLYSLTVPFGQKPNIDVTDAMVDQAWDAQRIFKEAEKFFVSV

In [37]:
status = be.set_protein_color_scheme(
    assembly_name=name, name=name,
    color_scheme=be.COLOR_SCHEME_ATOMS,
    palette_name='Set1', palette_size=2)

RuntimeError: Material 1 is not registered in the model

In [38]:
status = be.set_protein_color_scheme(
    assembly_name=name, name=name,
    color_scheme=be.COLOR_SCHEME_CHAINS,
    palette_name='Set1', palette_size=5)

## Visualization of amino acids sequence

In [ ]:
status = be.set_protein_amino_acid_sequence_as_ranges(
    assembly_name=name, name=name, amino_acid_ranges=[100,150])
status = be.set_protein_color_scheme(
    assembly_name=name, name=name,
    color_scheme=be.COLOR_SCHEME_AMINO_ACID_SEQUENCE,
    palette_name='Set1', palette_size=2)

## Visualization of glycosylation sites

In [ ]:
status = be.set_protein_color_scheme(
    assembly_name=name, name=name,
    color_scheme=be.COLOR_SCHEME_GLYCOSYLATION_SITE,
    palette_name='Set1', palette_size=2)

## Visualization of functional regions

In [ ]:
# RGB color palette for amino acid indices
grey = [0.5, 0.5, 0.5]
dark_green = [0.0, 0.5, 0.0]
light_green = [0.0, 1.0, 0.0]

region_indices_and_colors = [
    [   1, grey       ], [   5, grey],  [  30, light_green],  [  41, dark_green],
    [  82, light_green], [  84, grey],  [ 353, grey       ],  [ 357, grey      ],
    [ 800, grey       ]]

# Build a palette according to region colors
palette = list()
for index in range(len(region_indices_and_colors)-1):
    for i in range(region_indices_and_colors[index + 1][0] - 
                   region_indices_and_colors[index][0]):
        palette.append(region_indices_and_colors[index][1])

# Apply palette to other chains
status = be.set_protein_color_scheme(
    assembly_name=name, name=name, 
    color_scheme=be.COLOR_SCHEME_CHAINS,
    palette_name='Greys', palette_size=5)

# Apply palette to region color scheme, only for chain #2
status = be.set_protein_color_scheme(
    assembly_name=name, name=name, 
    color_scheme=be.COLOR_SCHEME_REGION, palette=palette,
    chain_ids=[2])

## Add glycans to protein

In [ ]:
glycan_name = name + '_Complex'
glycans = Sugars(
    rotation=Quaternion(0.707, 0.0, 0.0, 0.707),
    assembly_name=name, name=glycan_name, protein_name=name,
    source=os.path.join(complex_folder, '36.pdb'),
)
status = be.add_glycans(glycans)

## Materials

In [ ]:
status = be.apply_default_color_scheme(
    shading_mode=be.SHADING_MODE_PERLIN,
    user_parameter=0.03, specular_exponent=5, glossiness=0.1)

In [ ]:
status = be.apply_default_color_scheme(
    shading_mode=be.SHADING_MODE_DIFFUSE,
    user_parameter=1, specular_exponent=50)

## Rendering settings

In [ ]:
status = be.core_api().set_renderer(
    background_color=[96 / 255, 125 / 255, 139 / 255],
    current='bio_explorer',
    samples_per_pixel=1, subsampling=4, max_accum_frames=128)
params = be.core_api().BioExplorerRendererParams()
params.gi_samples = 1
params.gi_weight = 0.3
params.gi_distance = 5
params.shadows = 1.0
params.soft_shadows = 1.0
params.fog_start = 1500
params.fog_thickness = 1500
params.max_bounces = 1
status = be.core_api().set_renderer_params(params)

## Snapshot

In [ ]:
mm = MovieMaker(be)

In [ ]:
# mm.create_snapshot(
#     renderer='bio_explorer',
#     path='/tmp', base_name='ACE2_receptor',
#     size=[512,512], samples_per_pixel=64)